In [1]:
!pip install "sagemaker>=2.69.0" "transformers==4.12.3" --upgrade
# using older dataset due to incompatibility of sagemaker notebook & aws-cli with > s3fs and fsspec to >= 2021.10
!pip install  "datasets==1.13" --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached sagemaker-2.70.0.tar.gz (466 kB)
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.12.3-py3-none-any.whl (3.1 MB)
  Using cached huggingface_hub-0.2.1-py3-none-any.whl (61 kB)
  Using cached boto3-1.20.20-py3-none-any.whl (131 kB)
  Using cached s3transfer-0.5.0-py3-none-any.whl (79 kB)
  Using cached botocore-1.23.20-py3-none-any.whl (8.4 MB)
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/PyYAML-6.0.dist-info/METADATA'

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached datasets-1.13.0-py3-none-any.whl (285 kB)
  Using cached huggingface_hub-0.0.19-py3-none-any.whl (56 kB)
  Using cached fsspec-2021.11.1-py3-none-any.whl (132 kB)
  Using cached tqdm-4.62.3-py2.py3-none-any.whl (76 kB)


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/home/ec2-user/anaconda3/envs/pytorch_latest_p37/lib/python3.7/site-packages/PyYAML-6.0.dist-info/METADATA'



In [ ]:
%%capture
!pip install "sagemaker>=2.48.0" "transformers==4.6.1" "datasets[s3]==1.6.2" --upgrade
#!apt install git-lfs

In [ ]:
%%capture
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.rpm.sh | sudo bash
!sudo yum install git-lfs -y
!git lfs install

In [1]:
import sagemaker

In [2]:
# assert sagemaker.__version__ >= "2.69.0"

In [3]:
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker
from sagemaker import get_execution_role

In [4]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

### Push to hub

In [11]:
import os
import tarfile
from sagemaker.s3 import S3Downloader

local_path = 'my_roberta_base_emotion_results'

os.makedirs(local_path, exist_ok = True)

# download model from S3
S3Downloader.download(
    s3_uri='s3://emotion-class-197614225699/training_output/emotion-training-2021-12-03-20-58-03-225/output/output.tar.gz', # s3 uri where the trained model is located
    local_path=local_path, # local path where *.targ.gz is saved
    sagemaker_session=sess # sagemaker session used for training the model
)

# unzip model
tar = tarfile.open(f"{local_path}/output.tar.gz", "r:gz")
tar.extractall(path=local_path)
tar.close()

In [5]:
local_path = 'my_roberta_base_emotion'

In [11]:
import json


MODEL_CARD_TEMPLATE = """
---
language: en
widget:
- text: "I am really upset that I have to call up to three times to the number on the back of my insurance card for my call to be answer"
tags:
- sagemaker
- roberta-base
- text classification
license: apache-2.0
datasets:
- emotion
model-index:
- name: sagemaker-roberta-base-emotion
  results:
  - task: 
      name: Multi Class Text Classification
      type: text-classification
    dataset:
      name: "emotion" 
      type: emotion
    metrics:
       - name: Validation Accuracy
         type: accuracy
         value: 94.1
       - name: Validation F1
         type: f1
         value: 94.13
  
---
## roberta-base

This model is a fine-tuned model that was trained using Amazon SageMaker and the new Hugging Face Deep Learning container.
- Problem type: Multi Class Text Classification (emotion detection).

It achieves the following results on the evaluation set:
- Loss: 0.1613253802061081
- f1: 0.9413321705151999

## Hyperparameters
```json
{
    "epochs": 10,
    "train_batch_size": 16,
    "learning_rate": 3e-5, 
    "weight_decay":0.01,
    "load_best_model_at_end": true,
    "model_name":"roberta-base",
    "do_eval": True,
    "load_best_model_at_end":True
}
```
## Validation Metrics
| key | value |
| --- | ----- |
| eval_accuracy  | 0.941 |
| eval_f1 | 0.9413321705151999 |
| eval_loss | 0.1613253802061081|
| eval_recall | 0.941 |
| eval_precision | 0.9419519436781406 |

"""

In [12]:
# Generate model card (todo: add more data from Trainer)
model_card = MODEL_CARD_TEMPLATE
with open(f"{local_path}/README.md", "w") as f:
    f.write(model_card)

In [13]:
model_card 

'\n---\nlanguage: en\nwidget:\n- text: "I am really upset that I have to call up to three times to the number on the back of my insurance card for my call to be answer"\ntags:\n- sagemaker\n- roberta-base\n- text classification\nlicense: apache-2.0\ndatasets:\n- emotion\nmodel-index:\n- name: sagemaker-roberta-base-emotion\n  results:\n  - task: \n      name: Multi Class Text Classification\n      type: text-classification\n    dataset:\n      name: "emotion" \n      type: emotion\n    metrics:\n       - name: Validation Accuracy\n         type: accuracy\n         value: 94.1\n       - name: Validation F1\n         type: f1\n         value: 94.13\n  \n---\n## roberta-base\n\nThis model is a fine-tuned model that was trained using Amazon SageMaker and the new Hugging Face Deep Learning container.\n- Problem type: Multi Class Text Classification (emotion detection).\n\nIt achieves the following results on the evaluation set:\n- Loss: 0.1613253802061081\n- f1: 0.9413321705151999\n\n## Hyp

In [14]:
from getpass import getpass
from huggingface_hub import HfApi, Repository

hf_username = "Jorgeutd" # your username on huggingface.co
hf_email = "jll470@psu.edu" # email used for commit
#repository_name = f"{hyperparameters['model_name'].split('/')}-{['ade_corpus_v2, Ade_corpus_v2_classification']}" # repository name on huggingface.co
password = getpass("Enter your password:") # creates a prompt for entering password

# get hf token
token = HfApi().login(username=hf_username, password=password)

Enter your password:········


In [17]:
repository_name='Jorgeutd/sagemaker-roberta-base-emotion'

In [18]:
token

'vRUDIjihFvWfhtPrkgdiuBjBgAqkgDoHAbFUWzHMzKfzdXBoKuusoomyCMOhRwANmCFVaabGFbDBFcFLcpyhrYqFmSVpKmCklvdlgngZVWVKQPcZaGmXqKEasewuXCly'

In [20]:
local_path

'my_roberta_base_emotion'

In [21]:
# create a Repository instance
model_repo = Repository(use_auth_token=token,
                        clone_from="https://huggingface.co/Jorgeutd/sagemaker-roberta-base-emotion",
                        local_dir=local_path,
                        git_user=hf_username,
                        git_email=hf_email)

# push model to the hub
model_repo.push_to_hub()

print(f"https://huggingface.co/{hf_username}/{repository_name}")

In [22]:
print(f"https://huggingface.co/{hf_username}/{repository_name}")

https://huggingface.co/Jorgeutd/Jorgeutd/sagemaker-roberta-base-emotion


#### usage in HF hub 

In [23]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("Jorgeutd/sagemaker-roberta-base-emotion")

model = AutoModelForSequenceClassification.from_pretrained("Jorgeutd/sagemaker-roberta-base-emotion")

In [26]:
import torch
# If there"s a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print("There are %d GPU(s) available." % torch.cuda.device_count())
    print("We will use the GPU:", torch.cuda.get_device_name(0))
# If not...
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [27]:
import time

start_time = time.time()

from transformers import pipeline
classifier = pipeline('text-classification', tokenizer=tokenizer, model=model)


print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0002465248107910156 seconds ---


In [28]:
classifier('I am really upset that I have to call up to three times to the number on the back of my insurance card for my call to be answer')

[{'label': 'anger', 'score': 0.9984963536262512}]

### Load model from S3 or Local Path

In [32]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    model_data="s3://emotion-class-197614225699/training_output/emotion-training-2021-12-03-20-58-03-225/output/model.tar.gz",
    role=role,
   transformers_version = '4.12',           # the transformers version used in the training job
    pytorch_version      = '1.9',           # the pytorch_version version used in the training job
    py_version           = 'py38',            # the python version used in the training job
)

In [33]:
huggingface_model 

### Deploy model

In [35]:
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.g4dn.xlarge",
   endpoint_name='emotiondetection'
)

------------!

### Load test dataset for inference requests

In [36]:
import pandas as pd

In [37]:
import time

start_time = time.time()

import botocore
from datasets.filesystems import S3FileSystem
from datasets import load_from_disk

s3_session = botocore.session.Session()
s3 = S3FileSystem(session=s3_session)
dataset_test = load_from_disk('s3://emotion-class-197614225699/processing_output/test_data', fs=s3)
df_test = pd.DataFrame(dataset_test)

print("--- %s seconds ---" % (time.time() - start_time))

--- 2.056455612182617 seconds ---


In [38]:
df_test.columns

Index(['attention_mask', 'input_ids', 'label', 'text'], dtype='object')

In [39]:
from pandas import option_context

with option_context('display.max_rows', None,'display.max_colwidth', 2000,'display.max_columns',50):
              display(df_test.head(50))

,attention_mask,input_ids,label,text
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 757, 2157, 1195, 34485, 98, 4356, 45, 182, 8263, 235, 122, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]",0,im feeling rather rotten so im not very ambitious right now
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 757, 18796, 127, 5059, 142, 939, 619, 43816, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]",0,im updating my blog because i feel shitty
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 118, 393, 146, 69, 2559, 31, 162, 142, 939, 218, 326, 655, 236, 69, 7, 619, 101, 939, 475, 20085, 19, 69, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]",0,i never make her separate from me because i don t ever want her to feel like i m ashamed with her
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 118, 314, 19, 127, 20437, 15259, 9, 1275, 8, 5718, 30892, 7418, 223, 127, 3124, 2157, 2829, 55, 7168, 87, 77, 939, 2035, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]",1,i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 118, 21, 2157, 10, 410, 25876, 77, 939, 222, 42, 65, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...]",0,i was feeling a little vain when i did this one
5,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...]","[0, 118, 17672, 1656, 88, 10, 2

In [43]:
{"label2id": {
    "anger": "3",
    "fear": "4",
    "joy": "1",
    "love": "2",
    "sadness": "0",
    "surprise": "5"
  }
}

{'label2id': {'anger': '3',
  'fear': '4',
  'joy': '1',
  'love': '2',
  'sadness': '0',
  'surprise': '5'}}

In [44]:
df_test.iloc[3]['text']

'i left with my bouquet of red and yellow tulips under my arm feeling slightly more optimistic than when i arrived'

In [45]:
emotion_input= {"inputs": df_test.iloc[3]['text']}
predictor.predict(emotion_input)

[{'label': 'joy', 'score': 0.9993501305580139}]

In [48]:
df_test.iloc[8]['text']

'i like to have the same breathless feeling as a reader eager to see what will happen next'

In [49]:
emotion_input= {"inputs": df_test.iloc[8]['text']}
predictor.predict(emotion_input)

[{'label': 'joy', 'score': 0.9993797540664673}]

In [50]:
df_test.iloc[9]['text']

'i jest i feel grumpy tired and pre menstrual which i probably am but then again its only been a week and im about as fit as a walrus on vacation for the summer'

In [51]:
emotion_input= {"inputs": df_test.iloc[9]['text']}
predictor.predict(emotion_input)

[{'label': 'anger', 'score': 0.9988406300544739}]

In [52]:
df_test.iloc[15]['text']

'i was feeling as heartbroken as im sure katniss was'

In [53]:
emotion_input= {"inputs": df_test.iloc[15]['text']}
predictor.predict(emotion_input)

[{'label': 'sadness', 'score': 0.9993054866790771}]

In [54]:
df_test.iloc[27]['text']

'i stole a book from one of my all time favorite authors and now i feel like a rotten person'

In [55]:
emotion_input= {"inputs": df_test.iloc[27]['text']}
predictor.predict(emotion_input)

[{'label': 'sadness', 'score': 0.9993051290512085}]

In [56]:
### Play with the model with made-up phrases

In [59]:
emotion_input= {"inputs": "I am really susprise for such a good customer service"}
predictor.predict(emotion_input)

[{'label': 'joy', 'score': 0.9987189769744873}]

In [63]:
emotion_input= {"inputs": "I've never had such a bad day in my life because of the terrible service"}
predictor.predict(emotion_input)

[{'label': 'sadness', 'score': 0.9765757322311401}]

In [62]:
emotion_input= {"inputs": "I fear the delay time on delivery"}
predictor.predict(emotion_input)

[{'label': 'fear', 'score': 0.9885130524635315}]

In [64]:
#### Finally, we delete the inference endpoint.

predictor.delete_endpoint()